<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train10_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train10 = pd.read_csv(r'/content/train_data10.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train10['Target'] = train10['Target'].replace(['Others'],'Invalid')


In [ ]:
train10['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train10_Xtfidf = tfidf_vect.fit_transform(train10['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train10_Xtfidf,train10['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6603982300884956
                  precision    recall  f1-score   support

        Analysis       0.52      0.34      0.41       131
      Conclusion       0.78      0.40      0.53        45
           Facts       0.74      0.86      0.80       485
         Invalid       0.47      0.60      0.53       123
           Issue       0.57      0.40      0.47        53
Rule/Law/Holding       0.65      0.30      0.41        67

        accuracy                           0.66       904
       macro avg       0.62      0.48      0.52       904
    weighted avg       0.65      0.66      0.64       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 3, 5, 2, 2, 2, 5, 3, 2]
[0.9975637697831035, 0.9956917397470824, 0.9850473642376405, 0.9987989260185687, 0.9989135904035754, 0.9954245735314573, 0.9971457736510522, 0.9976565027805304, 0.9880095125646129, 0.9916180332622563, 0.9971917824573795, 0.9910238528432778, 0.9956174284625358, 0.996032962403671, 0.9979815605411663, 0.9975887913508585, 0.9884436282812256, 0.9973551569279617, 0.9926355391085008, 0.9955277962705943, 0.9983333137643484, 0.9954856868733662, 0.9992130402195266, 0.9986977671361498, 0.9896977067908468, 0.9895832476557075, 0.9956098902656375, 0.9936055100843854, 0.9937904721137646, 0.9963495497982022, 0.9955356342057663, 0.9822004626466155, 0.9953364465898693, 0.987924667023794, 0.9912260548300564, 0.9984494040764936, 0.9822504980943094, 0.9803931753476011, 0.9955932160283634, 0.9885872542345514, 0.997927373

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
3,2199,proper instruction court murder malice murder ...,Facts
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts


In [ ]:
frame_1 = [train10,unlabel_1]
train10_1 = pd.concat(frame_1)
len(train10_1)

3539

In [ ]:
x_train_1 = tfidf_vect.transform(train10_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train10_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6567796610169492
                  precision    recall  f1-score   support

        Analysis       0.49      0.33      0.40        93
      Conclusion       0.67      0.45      0.54        31
           Facts       0.72      0.86      0.79       377
         Invalid       0.53      0.57      0.55       107
           Issue       0.50      0.33      0.40        45
Rule/Law/Holding       0.64      0.33      0.43        55

        accuracy                           0.66       708
       macro avg       0.59      0.48      0.52       708
    weighted avg       0.64      0.66      0.64       708



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 3, 2, 3, 2, 3, 5, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 3, 2, 2, 2, 5, 3, 4, 2]
[0.998670503276418, 0.9990357777046353, 0.9888528744148133, 0.9965977703224234, 0.9986936171217878, 0.9982863428872342, 0.9989692369859018, 0.993756212835184, 0.9946114302812876, 0.9984102128143654, 0.9916552301634145, 0.989878569524415, 0.9910158507826065, 0.9975611113128652, 0.99628146187845, 0.9967678099118586, 0.992676777042977, 0.9981798705053964, 0.9990094145730171, 0.9990704727086327, 0.991709768683615, 0.9989901360679307, 0.9982924665638118, 0.999128121471244, 0.9963919975442893, 0.9987559484419162, 0.9960960553120597, 0.994217849609322, 0.9986896496561359, 0.9866852891862711, 0.9914697941509955, 0.9989127081065883, 0.9988407304494576, 0.9982536162601212, 0.9988086933721947, 0.9951727224430134, 0.9978509207441297, 0.9965684847062753, 0.9966464932733043, 0.9990067109239986, 0.9979333199486473, 0.998945614498025, 0.989559326374356, 0.

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [ ]:
frame_2 = [train10_1,unlabel_2]
train10_2 = pd.concat(frame_2)
len(train10_2)

3588

In [ ]:
x_train_2 = tfidf_vect.transform(train10_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train10_2['Target'],test_size=0.24,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6194895591647331
                  precision    recall  f1-score   support

        Analysis       0.52      0.32      0.40       136
      Conclusion       0.74      0.30      0.43        46
           Facts       0.65      0.88      0.75       417
         Invalid       0.51      0.50      0.50       150
           Issue       0.78      0.27      0.40        52
Rule/Law/Holding       0.66      0.34      0.45        61

        accuracy                           0.62       862
       macro avg       0.64      0.44      0.49       862
    weighted avg       0.62      0.62      0.59       862



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 3, 3, 2, 2, 2, 2, 3, 3, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 3, 3, 1, 5, 2, 5, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 0, 2, 2, 2, 4, 2, 2, 2, 2, 2]
[0.9930304233396382, 0.9970837251976017, 0.9976477547820372, 0.9981703349382595, 0.9888541009832401, 0.9856168595952568, 0.9977274132015113, 0.9974549939954827, 0.9826447618780344, 0.9985261949780468, 0.998383709332281, 0.998511376368448, 0.9810706063308443, 0.9986024529316304, 0.9877573857192457, 0.9981415206491833, 0.9912420673569335, 0.9965957194097201, 0.9959023926677874, 0.998137766765913, 0.9806807518117812, 0.9804537440362676, 0.9983261751998599, 0.9924575020143264, 0.9874515365133456, 0.9969360985904333, 0.9966480556534693, 0.9980297610565207, 0.9839225396931355, 0.9957563857888256, 0.996110375618039, 0.9920471181636656, 0.9971185097685025, 0.9976272982695834, 0.9850956174234936, 0.9962047519928027, 0.9980748951549333, 0.9931167725614277, 0.9962274681900358, 0.9980655

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train10_2,unlabel_3]
train10_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train10_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train10_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6489071038251366
                  precision    recall  f1-score   support

        Analysis       0.41      0.30      0.35        96
      Conclusion       0.73      0.41      0.52        39
           Facts       0.72      0.85      0.78       392
         Invalid       0.54      0.59      0.56       121
           Issue       0.50      0.38      0.43        34
Rule/Law/Holding       0.64      0.28      0.39        50

        accuracy                           0.65       732
       macro avg       0.59      0.47      0.51       732
    weighted avg       0.64      0.65      0.63       732



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 0, 3, 2, 5, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 3, 2, 3, 0, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2]
[0.9947927952120709, 0.9922317973114981, 0.9878514244042013, 0.9831798264789372, 0.9980536747266645, 0.9972865387310099, 0.9976776599714573, 0.9973776227687414, 0.986051091357452, 0.9988721027343721, 0.9985127432893122, 0.9963435337787794, 0.9971106913321891, 0.9975957441562014, 0.9941362362923144, 0.9988462297749349, 0.9983362583421823, 0.9984273379934862, 0.998443039080071, 0.9926898904244457, 0.9812606360573655, 0.9961394285204951, 0.9986601410561032, 0.9982774030702535, 0.9987522634971827, 0.9989784885575902, 0.9976292759793634, 0.998423347787355, 0.9987886760908469, 0.9983348067457569, 0.9986405134934695, 0.9954770082295563, 0.9984290503201688, 0.9866917442891964, 0.9968434782464667, 0.9962476370475055, 0.9845436457344883, 0.9974659107224642, 0.9894239220868654, 0.9954788888688589, 0.9987708314

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train10_3,unlabel_4]
train10_4 = pd.concat(frame_4)
len(train10_4)
x_train_4 = tfidf_vect.transform(train10_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train10_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.663681592039801
                  precision    recall  f1-score   support

        Analysis       0.51      0.32      0.39       146
      Conclusion       0.74      0.47      0.57        53
           Facts       0.71      0.87      0.78       525
         Invalid       0.55      0.59      0.57       160
           Issue       0.67      0.38      0.48        53
Rule/Law/Holding       0.65      0.38      0.48        68

        accuracy                           0.66      1005
       macro avg       0.64      0.50      0.55      1005
    weighted avg       0.65      0.66      0.64      1005



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 2, 3, 1, 2, 5, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 1, 2, 2, 2, 3, 2, 5, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 2, 3, 2, 2]
[0.9989454458118044, 0.9988944453305719, 0.9933615237121325, 0.9973354791274438, 0.986895444195542, 0.9986342592542171, 0.9883234778056826, 0.9984790835192252, 0.9988944453305719, 0.9949104934953177, 0.9985154006655227, 0.9898952515785389, 0.9854225397478448, 0.9917963187395854, 0.9938739676392444, 0.9951784455298043, 0.999192273301865, 0.9988426974546482, 0.9955161242739858, 0.9982094141867911, 0.9908332636928052, 0.9960483240041882, 0.9987951972762629, 0.9987821146161957, 0.9987754087179749, 0.9988944453305719, 0.9985687778666963, 0.9978136466995357, 0.994940930614204, 0.9935252019006598, 0.9986831175280605, 0.9968597862676507, 0.999192273301865, 0.9859080629868926, 0.99648132251667, 0.9976142464806979, 0.9972622613612293, 0.9963741779447272, 0.9931443404591704, 0.9938776052778427, 0.9956012972096683, 0.99

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train10_4,unlabel_5]
train10_5 = pd.concat(frame_5)
len(train10_5)
x_train_5 = tfidf_vect.transform(train10_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train10_5['Target'],test_size=0.24,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6916299559471366
                  precision    recall  f1-score   support

        Analysis       0.53      0.42      0.47       124
      Conclusion       0.85      0.52      0.65        44
           Facts       0.72      0.89      0.80       485
         Invalid       0.61      0.54      0.57       141
           Issue       0.75      0.44      0.56        54
Rule/Law/Holding       0.73      0.32      0.44        60

        accuracy                           0.69       908
       macro avg       0.70      0.52      0.58       908
    weighted avg       0.69      0.69      0.67       908



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))


Accuracy 0.7658349328214972


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7612144991278619, 0.6608741222191719, 0.6990095198709868, None)
